In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Top 5 products by revenue
#events = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv", header=True, inferSchema=True)
revenue = events.filter(F.col("event_type") == "purchase") \
    .groupBy("product_id", "brand") \
    .agg(F.sum("price").alias("revenue")) \
    .orderBy(F.desc("revenue")).limit(5)
display(revenue)

# Running total per user
window = Window.partitionBy("user_id").orderBy("event_time")
cumulative_events = events.withColumn("cumulative_events", F.count("*").over(window))
display(cumulative_events)

# Conversion rate by category
conversion_rate = events.groupBy("category_code") \
    .pivot("event_type") \
    .count() \
    .withColumn(
        "conversion_rate",
        F.col("purchase") / F.col("view") * 100
    )
display(conversion_rate)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import sum

data = [
    (1, "A", 100),
    (1, "A", 200),
    (2, "B", 400),
    (2, "B", 200),
    (2, "B", 200),
    (3, "C", 300)
]

df = spark.createDataFrame(data, ["user_id", "brand", "amount"])

w = Window.partitionBy("brand")
w1= Window.partitionBy("user_id").orderBy("amount")

df.withColumn("brand_total", sum("amount").over(w1)).show()


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *

w= Window.partitionBy("user_id").orderBy("amount")
df.withColumn("brand_total", sum("amount").over(w)).show()

w1= Window.partitionBy("user_id")
df.withColumn("brand_total", sum("amount").over(w1)).show()

w2 = Window.partitionBy("brand").orderBy("amount")
df.withColumn("rank", rank().over(w2)).show()
df.withColumn("dense_rank", dense_rank().over(w2)).show()
df.withColumn("percent_rank", percent_rank().over(w2)).show()